# Assignment 5, Question 6: Data Transformation

**Points: 20**

Transform and engineer features from the clinical trial dataset.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import utilities
from q3_data_utils import load_data, clean_data, transform_types, create_bins, fill_missing

# df = load_data('data/clinical_trial_raw.csv')
df = load_data('output/q5_cleaned_data.csv')
print(f"Loaded {len(df)} patients")

# Prewritten visualization functions for transformation analysis
def plot_distribution(series, title, figsize=(10, 6)):
    """
    Create a histogram of a numeric series.
    
    Args:
        series: pandas Series with numeric data
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.hist(bins=30)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

def plot_value_counts(series, title, figsize=(10, 6)):
    """
    Create a bar chart of value counts.
    
    Args:
        series: pandas Series with value counts
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Loaded 10000 patients


## Part 1: Type Conversions (5 points)

1. Convert 'enrollment_date' to datetime using the `transform_types()` utility
2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
3. Ensure all numeric columns are proper numeric types
4. Display the updated dtypes

In [2]:
# TODO: Type conversions
# Make a copy
df_transformed = df.copy()
display(df_transformed['site'].unique())
# 1. Use transform_types() to convert enrollment_date to datetime
df_transformed = transform_types(df_transformed, {'enrollment_date':'datetime'})

# 2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
df_transformed = transform_types(df_transformed, {'site':'category', 'intervention_group':'category', 'sex':'category'})

# 3. Ensure all numeric columns are proper numeric types
df_transformed = transform_types(df_transformed, {'site':'category', 'intervention_group':'category', 'sex':'category'})

# 4. Display the updated dtypes using df.dtypes
display(df_transformed.dtypes)
# display(df_transformed.head())


array(['site b', 'Site A', 'SITE C', 'Site D', 'site e', 'site a',
       'SITE B', 'SITE A', 'SITE E', 'site d', 'Site E', 'site c',
       'Site C', '  SITE D  ', '  Site B  ', 'Site_D', 'SITE D',
       '  site a  ', 'Site B', '  site e  ', '  SITE B  ', 'Site  A',
       '  Site  A  ', '  site d  ', '  site b  ', '  site c  ',
       '  Site E  ', '  SITE E  ', '  SITE C  ', '  SITE A  ',
       '  Site A  ', '  Site_D  ', '  Site C  ', '  Site D  '],
      dtype=object)

patient_id                    object
age                          float64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                   object
adherence_pct                float64
dropout                       object
dtype: object

## Part 2: Feature Engineering (8 points)

Create these new calculated columns:

1. `cholesterol_ratio` = cholesterol_ldl / cholesterol_hdl
2. `bp_category` = categorize systolic BP:
   - 'Normal': < 120
   - 'Elevated': 120-129
   - 'High': >= 130
3. `age_group` using `create_bins()` utility:
   - Bins: [0, 40, 55, 70, 100]
   - Labels: ['<40', '40-54', '55-69', '70+']
4. `bmi_category` using standard BMI categories:
   - Underweight: <18.5
   - Normal: 18.5-24.9
   - Overweight: 25-29.9
   - Obese: >=30

In [3]:
# TODO: Calculate cholesterol ratio
df_transformed['cholesterol_ratio'] = df_transformed['cholesterol_ldl']/df_transformed['cholesterol_hdl']

In [4]:
# TODO: Categorize blood pressure
df_transformed = create_bins(df_transformed, 'systolic_bp', [0,120,130,999], ['Normal', 'Elevated', 'High'], 'bp_category')
df_transformed

,patient_id,age,sex,bmi,enrollment_date,systolic_bp,diastolic_bp,cholesterol_total,cholesterol_hdl,cholesterol_ldl,glucose_fasting,site,intervention_group,follow_up_months,adverse_events,outcome_cvd,adherence_pct,dropout,cholesterol_ratio,bp_category
0,P00001,80.0,F,29.3,2022-05-01,123.0,80.0,120.0,55.0,41.0,118.0,site b,Control,20,0,No,24.0,No,0.745455,Elevated
1,P00002,80.0,Female,29.3,2022-01-06,139.0,81.0,206.0,58.0,107.0,79.0,Site A,CONTROL,24,0,No,77.0,No,1.844828,High
2,P00003,82.0,Female,29.3,2023-11-04,123.0,86.0,172.0,56.0,82.0,77.0,SITE C,treatment b,2,0,Yes,70.0,No,1.464286,Elevated
3,P00004,95.0,Female,25.4,2022-08-15,116.0,77.0,200.0,56.0,104.0,115.0,Site D,treatment b,17,0,No,62.0,No,1.857143,Normal
4,P00005,95.0,M,25.4,2023-04-17,97.0,71.0,185.0,78.0,75.0,113.0,site e,Treatmen A,9,0,yes,62.0,Yes,0.961538,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,P09996,72.0,Male,23.2,2022-04-11,122.0,73.0,182.0,54.0,92.0,97.0,site c,TREATMENT B,21,0,No,20.0,No,1.703704,Elevated
9996,P09997,100.0,Female,28.9,2023-02-10,124.0,78.0,157.0,56.0,70.0,102.0,Site C,control,11,0,No,57.0,No,1.250000,Elevated
9997,P09998,78.0,F,23.8,2023-11-05,110.0,63.0,154.0,69.0,71.0,114.0,Site C,treatment a,2,1,No,77.0,No,1.028986,Normal
9998,P09999,86.0,F,27.0,2022-08-27,139.0,98.0,196.0,38.0,119.0,126.0,SITE A,CONTROL,16,0,no,63.0,No,3.131579,High


**Note:** The `create_bins()` function has an optional `new_column` parameter. If you don't specify it, the new column will be named `{original_column}_binned`. You can use `new_column='age_group'` to give it a custom name.


In [5]:
# TODO: Create age groups
df_transformed = create_bins(df_transformed, 'age', [0, 40, 55, 70, 100], ['<40', '40-54', '55-69', '70+'], 'age_group')
df_transformed.dtypes

patient_id                    object
age                          float64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                   object
adherence_pct                float64
dropout                       object
cholesterol_ratio            float64
bp_category                 category
age_group                   category
dtype: object

In [6]:
# TODO: Create BMI categories
df_transformed = create_bins(df_transformed, 'bmi', [0, 18.5, 25, 30, 999], ['Underweight', 'Normal', 'Overweight', 'Obese'], 'bmi_category')


## Part 3: String Cleaning (2 points)

If there are any string columns that need cleaning:
1. Convert to lowercase
2. Strip whitespace
3. Replace any placeholder values

In [7]:
# TODO: String cleaning
string_columns = df_transformed.select_dtypes(include=['object', 'category']).columns

for i in string_columns:
    df_transformed[i] = df_transformed[i].str.lower()
    df_transformed[i] = df_transformed[i].str.strip()
    if i != 'age_group':
        df_transformed[i] = df_transformed[i].str.replace(r'[-_]|\s+', ' ', regex=True)

    # Handling placeholder values
    if i == 'sex':
        df_transformed[i] = df_transformed[i].replace({'f': 'female', 'm': 'male'})
    if i == 'intervention_group':
        df_transformed[i] = df_transformed[i].replace({'contrl': 'control', 'treatmen a': 'treatment a', 'treatmenta': 'treatment a'})   

    # Validate
    display(df_transformed[i].unique())

display(df_transformed.head())

array(['p00001', 'p00002', 'p00003', ..., 'p09998', 'p09999', 'p10000'],
      shape=(10000,), dtype=object)

array(['female', 'male'], dtype=object)

array(['site b', 'site a', 'site c', 'site d', 'site e'], dtype=object)

array(['control', 'treatment b', 'treatment a'], dtype=object)

array(['no', 'yes'], dtype=object)

array(['no', 'yes'], dtype=object)

array(['elevated', 'high', 'normal'], dtype=object)

array(['70+', '55-69', '40-54'], dtype=object)

array(['overweight', 'normal', 'obese', 'underweight'], dtype=object)

,patient_id,age,sex,bmi,enrollment_date,systolic_bp,diastolic_bp,cholesterol_total,cholesterol_hdl,cholesterol_ldl,...,intervention_group,follow_up_months,adverse_events,outcome_cvd,adherence_pct,dropout,cholesterol_ratio,bp_category,age_group,bmi_category
0,p00001,80.0,female,29.3,2022-05-01,123.0,80.0,120.0,55.0,41.0,...,control,20,0,no,24.0,no,0.745455,elevated,70+,overweight
1,p00002,80.0,female,29.3,2022-01-06,139.0,81.0,206.0,58.0,107.0,...,control,24,0,no,77.0,no,1.844828,high,70+,overweight
2,p00003,82.0,female,29.3,2023-11-04,123.0,86.0,172.0,56.0,82.0,...,treatment b,2,0,yes,70.0,no,1.464286,elevated,70+,overweight
3,p00004,95.0,female,25.4,2022-08-15,116.0,77.0,200.0,56.0,104.0,...,treatment b,17,0,no,62.0,no,1.857143,normal,70+,overweight
4,p00005,95.0,male,25.4,2023-04-17,97.0,71.0,185.0,78.0,75.0,...,treatment a,9,0,yes,62.0,yes,0.961538,normal,70+,overweight


## From Q4 Part 4: Value Counts and Grouping (5 points)

Moving it here because the dataset isn't cleaned until now (after talking to the professor). 

1. Get value counts for the 'site' column
2. Get value counts for the 'intervention_group' column  
3. Create a crosstab of site vs intervention_group
4. Calculate mean age by site
5. Save the site value counts to `output/q4_site_counts.csv`

In [8]:
# TODO: Value counts and analysis
site_counts = df_transformed.groupby('site')['site'].agg(['count'])
display(site_counts)

inter_counts = df_transformed.groupby('intervention_group')['intervention_group'].agg(['count'])
display(inter_counts)

crosstabs = pd.pivot_table(df_transformed, values='patient_id',
                            index='intervention_group', columns='site',
                            aggfunc='count')
display(crosstabs)

site_means = df_transformed.groupby('site')['age'].agg(['mean'])
display(site_means)

# TODO: Save output
site_counts.to_csv('output/q4_site_counts.csv')

,count
site,
site a,2956
site b,2453
site c,2073
site d,1501
site e,1017


,count
intervention_group,
control,3337
treatment a,3321
treatment b,3342


site,site a,site b,site c,site d,site e
intervention_group,,,,,
control,1023,780,709,490,335
treatment a,964,834,670,522,331
treatment b,969,839,694,489,351


,mean
site,
site a,81.201962
site b,80.408479
site c,80.655572
site d,80.718854
site e,80.339233


## Part 4: One-Hot Encoding (5 points)

Create dummy variables for categorical columns:
1. One-hot encode 'intervention_group' using `pd.get_dummies()`
2. One-hot encode 'site'
3. Drop the original categorical columns
4. Show the new shape and column names

In [9]:
# TODO: One-hot encoding
inter_dummies = pd.get_dummies(df_transformed['intervention_group'])
df_transformed = pd.concat([df_transformed, inter_dummies], axis=1)

site_dummies = pd.get_dummies(df_transformed['site'])
df_transformed = pd.concat([df_transformed, site_dummies], axis=1)

df_transformed = df_transformed.drop('site', axis=1)
df_transformed = df_transformed.drop('intervention_group', axis=1)
print(f"New dataframe shape: {df_transformed.shape}")
print(f"===== New dataframe columns =====")
display(df_transformed.columns)

New dataframe shape: (10000, 28)
===== New dataframe columns =====


Index(['patient_id', 'age', 'sex', 'bmi', 'enrollment_date', 'systolic_bp',
       'diastolic_bp', 'cholesterol_total', 'cholesterol_hdl',
       'cholesterol_ldl', 'glucose_fasting', 'follow_up_months',
       'adverse_events', 'outcome_cvd', 'adherence_pct', 'dropout',
       'cholesterol_ratio', 'bp_category', 'age_group', 'bmi_category',
       'control', 'treatment a', 'treatment b', 'site a', 'site b', 'site c',
       'site d', 'site e'],
      dtype='object')

## Part 5: Save Transformed Data

Save the fully transformed dataset to `output/q6_transformed_data.csv`

In [10]:
# TODO: Save transformed data
df_transformed.to_csv('output/q6_transformed_data.csv', index=False)